<a href="https://colab.research.google.com/github/Bhavya-Agrawal/Hand-Gesture-Recognition/blob/master/vgg16_hg_with_augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing dataset from Kaggle + Unzip the folders

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"parulchutaniphd","key":"a62ab73f77a8270aeafb2235e6a7aef2"}'}

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d prathumarikeri/indian-sign-language-isl

 97% 273M/281M [00:01<00:00, 142MB/s]
100% 281M/281M [00:01<00:00, 157MB/s]


In [ ]:
from zipfile import ZipFile
isl = "/content/indian-sign-language-isl.zip"
with ZipFile(isl,'r') as zip:
  zip.extractall();
  print("Dataset uploaded successfully !!!")

Dataset uploaded successfully !!!


# Collecting the dataset

In [ ]:
from glob import glob

In [ ]:
all_images = glob('/content/Indian/*/*.jpg', recursive=True)

In [ ]:
len(all_images)

42745

In [ ]:
unique_labels = []

total_images = []
total_labels = []
images = glob('/content/Indian/*/*.jpg', recursive=True)

for image in images:
  total_images.append(image)

  label = image.split('/')[-2]
  total_labels.append(label)

  if label not in unique_labels:
    unique_labels.append(label)

In [ ]:
len(unique_labels), len(total_labels), len(total_images)

(35, 42745, 42745)

In [ ]:
pip install split-folders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(total_images, columns=['Images'])
df['Labels'] = total_labels

# Train-Val-Test Folders


In [ ]:
import splitfolders

In [ ]:
splitfolders.ratio('/content/Indian', 
                   output = 'ISL',
                   seed = 22, ratio = (.7,.2,.1))

Copying files: 42745 files [00:05, 7157.34 files/s]


# Operations on Images

In [ ]:
# converting RGB to grayscale images and storing the result in the same folder structure
import cv2
from skimage import io
import os
import shutil
folder_list = ['/content/ISL/train', '/content/ISL/val']
# folder_list = [train_generator, validation_generator]

for current_folder in folder_list:
  main_folder = current_folder
  for image_folder in os.listdir(main_folder):
    internal_folder = os.path.join(main_folder, image_folder)
    for image in os.listdir(internal_folder):
      image_path = os.path.join(internal_folder, image)

      # read image
      frame = cv2.imread(image_path) 

      # Resize the images 200*200
      # resized_img = cv2.resize(frame, (200, 200)) 

      # convert image to grayscale     
      gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

      # apply gaussian blur to the image
      blur = cv2.GaussianBlur(gray,(5,5),2)
      
      # apply boundary to the actual character in the image
      th3 = cv2.adaptiveThreshold(blur,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY_INV,11,2)

      ret, res = cv2.threshold(th3, 0, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)

      # replace the existing colored image with above filtered image in the same location    
      if os.path.exists(image_path):
        os.remove(image_path)
      cv2.imwrite(image_path, res)


# Dataframe for training & validation images

In [ ]:
training_images = glob('/content/ISL/train/*/*.jpg')

training_labels = []

for image in training_images:
  label = image.split('/')[-2]
  training_labels.append(label)


In [ ]:
training_df = pd.DataFrame(training_images, columns=['Images'])
training_df['Labels'] = training_labels

In [ ]:
validation_images = glob('/content/ISL/val/*/*.jpg')

validation_labels = []

for image in validation_images:
  label = image.split('/')[-2]
  validation_labels.append(label)

In [ ]:
validation_df = pd.DataFrame(validation_images, columns=['Images'])
validation_df['Labels'] = validation_labels

In [ ]:
test_images = glob('/content/ISL/val/*/*.jpg')

test_labels = []

for image in test_images:
  label = image.split('/')[-2]
  test_labels.append(label)

In [ ]:
test_df = pd.DataFrame(test_images, columns=['Test Images'])
test_df['Test Labels'] = test_labels

In [ ]:
# frames = [df1, df2]
  
df = pd.concat([training_df, validation_df])

In [ ]:
df.sample(5)

,Images,Labels
20196,/content/ISL/train/X/84.jpg,X
20720,/content/ISL/train/F/399.jpg,F
11036,/content/ISL/train/V/760.jpg,V
1746,/content/ISL/val/3/209.jpg,3
1994,/content/ISL/val/A/377.jpg,A


In [ ]:
# images = np.array(images)

# # images = images.astype('float32') / 255.0
# images.shape

# Data Augmentation

In [ ]:
import keras
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
# Data augmentation step
training_data_generator = ImageDataGenerator(rotation_range=20,
                                             width_shift_range=0.2,
                                             height_shift_range=0.2,
                                             horizontal_flip=True,
                                             validation_split=0.2,
                                             rescale = 1/255)

test_data_generation = ImageDataGenerator(rescale=1/255)

In [ ]:

train_generator = training_data_generator.flow_from_directory('/content/ISL/train', 
                                                               target_size=(150, 150), 
                                                               batch_size=20,
                                                               class_mode='binary')

validation_generator = training_data_generator.flow_from_directory('/content/ISL/val',
                                                                target_size=(150, 150),
                                                                batch_size=20,
                                                                class_mode='binary')
test_generator = test_data_generation.flow_from_directory('/content/ISL/test',
                                                                target_size=(150, 150),
                                                                batch_size=20,
                                                                class_mode='binary')

Found 29919 images belonging to 35 classes.
Found 8547 images belonging to 35 classes.
Found 4279 images belonging to 35 classes.


# Operations on Labels

In [ ]:
import numpy as np

In [ ]:
labels = np.array(df['Labels'])

In [ ]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
y=df['Labels'].values
print(y)


y_labelencoder = LabelEncoder ()
y = y_labelencoder.fit_transform (y)
print (y)

['G' 'G' 'G' ... 'S' 'S' 'S']
[15 15 15 ... 27 27 27]


In [ ]:
y=y.reshape(-1,1)

In [ ]:
onehotencoder = OneHotEncoder(categories='auto')
# onehotencoder = OneHotEncoder(categorical_features=[0])  #Converted  scalar output into vector output where the correct class will be 1 and other will be 0
Y= onehotencoder.fit_transform(y)
Y.shape  #(40, 2)

(38466, 35)

# Creating VGG 16 Model

In [ ]:
import numpy as np

from keras.models import Model
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Input
from keras.layers import Conv2D
from keras.layers import MaxPooling2D

import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import os

In [ ]:
def VGG16_hand_gestures(input_tensor=None,classes=35):    
   
    img_rows, img_cols = 150, 150   # by default size is 224,224
    img_channels = 3

    img_dim = (img_rows, img_cols, img_channels)
   
    img_input = Input(shape=img_dim)
    
    # Block 1
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1')(img_input)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

    # Block 2
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

    # Block 3
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv2')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv3')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)

    # Block 4
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv1')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv2')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv3')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)

    # Block 5
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv1')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv2')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv3')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool')(x)

    
    # Classification block
    x = Flatten(name='flatten')(x)
    x = Dense(4096, activation='relu', name='fc1')(x)
    x = Dense(4096, activation='relu', name='fc2')(x)
    x = Dense(35, activation='softmax', name='predictions')(x)

    # Create model.
   
     
    model = Model(inputs = img_input, outputs = x, name='VGG16_hand_gestures')

    model.compile(optimizer='adam', loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
    
    print(model.summary())
    return model

In [ ]:
model = VGG16_hand_gestures(classes = 35)

Model: "VGG16_hand_gestures"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 150, 150, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 150, 150, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 75, 75, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 75, 75, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 75, 75, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 37, 37, 128

In [ ]:
h = model.fit_generator(
    train_generator,
    epochs=20,
    validation_data = validation_generator
)

Epoch 1/20


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  after removing the cwd from sys.path.


1496/1496 [==============================] - 248s 151ms/step - loss: 3.5630 - accuracy: 0.0330 - val_loss: 3.5544 - val_accuracy: 0.0338
Epoch 2/20
1496/1496 [==============================] - 224s 150ms/step - loss: 3.5547 - accuracy: 0.0335 - val_loss: 3.5542 - val_accuracy: 0.0338
Epoch 3/20
1496/1496 [==============================] - 222s 149ms/step - loss: 3.5547 - accuracy: 0.0337 - val_loss: 3.5542 - val_accuracy: 0.0338
Epoch 4/20
1496/1496 [==============================] - ETA: 0s - loss: 3.5546 - accuracy: 0.0333

KeyboardInterrupt: ignored